In [14]:
import pandas as pd

# Define the file name
file_name = "250409 Leng Survey - Regrouped and Combined.xlsx"

try:
    # Read the Excel file
    # By default, it reads the first sheet.
    # If your data is on a different sheet, use the sheet_name parameter.
    df = pd.read_excel(file_name)

    # Display the first few rows of the DataFrame to verify
    print("Successfully loaded the DataFrame!")
    print("First 5 rows:")
    print(df.head())

    # Display some information about the DataFrame
    print("\nDataFrame Info:")
    df.info()

    # Display the shape of the DataFrame (rows, columns)
    print("\nDataFrame Shape:")
    print(df.shape)

except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found.")
    print("Please ensure the file is in the same directory as your Jupyter notebook,")
    print("or provide the full path to the file.")
except Exception as e:
    print(f"An error occurred: {e}")

# If you know the specific sheet name you want to load, you can specify it:
# For example, if the sheet is named "Sheet1"
# df = pd.read_excel(file_name, sheet_name="Sheet1")

# Or by sheet index (0 for the first sheet, 1 for the second, etc.)
# df = pd.read_excel(file_name, sheet_name=0)

# If you want to load all sheets into a dictionary of DataFrames:
# all_sheets_df = pd.read_excel(file_name, sheet_name=None)
# for sheet_name, sheet_df in all_sheets_df.items():
#     print(f"\n--- Data from sheet: {sheet_name} ---")
#     print(sheet_df.head())

Successfully loaded the DataFrame!
First 5 rows:
  Which survey would you like to complete?  \
0               Physician associate survey   
1             Anaesthesia associate survey   
2               Physician associate survey   
3               Physician associate survey   
4             Anaesthesia associate survey   

  Do you work (currently or previously) as one of the following?  \
0                                Physician associate               
1                                                NaN               
2                                         Consultant               
3        Resident doctor, including foundation years               
4                                                NaN               

  Do you work (currently or previously) as one of the following?.1  \
0                                                NaN                 
1                              Anaesthesia associate                 
2                                                NaN   

In [15]:
import pandas as pd # Still need pandas for DataFrame operations

# Helper function to convert Excel column letters (e.g., 'A', 'B', 'AA', 'DN')
# to 0-based numerical index.
def excel_col_to_index(col_str):
    """Converts an Excel-style column string (A, B, AA, etc.) to a 0-based index."""
    index = 0
    power = 1
    for char in reversed(col_str.upper()): # Iterate from right to left
        index += (ord(char) - ord('A') + 1) * power
        power *= 26
    return index - 1 # Return 0-based index

# --- ANALYSIS CHUNK ---
# This chunk assumes 'df' (your DataFrame) is already loaded and available.

# Define the Excel column LETTERS you want to analyze
excel_column_letters_to_analyze = ['F', 'J', 'DN', 'DO', 'DP']
actual_column_names_to_analyze = []

# Check if 'df' exists and is a DataFrame
if 'df' in locals() and isinstance(df, pd.DataFrame):
    print("Attempting to map Excel column letters to DataFrame column names...")
    for col_letter in excel_column_letters_to_analyze:
        try:
            col_index = excel_col_to_index(col_letter)
            if 0 <= col_index < len(df.columns):
                actual_name = df.columns[col_index]
                actual_column_names_to_analyze.append(actual_name)
                print(f"  Excel column '{col_letter}' (0-indexed: {col_index}) maps to DataFrame column: '{actual_name}'")
            else:
                print(f"  ERROR: Excel column '{col_letter}' (index {col_index}) is out of bounds. "
                      f"DataFrame has {len(df.columns)} columns (0 to {len(df.columns)-1}).")
        except Exception as e:
            print(f"  ERROR: Could not process Excel column letter '{col_letter}'. Error: {e}")

    if not actual_column_names_to_analyze:
        print("\nNo valid DataFrame columns were identified from the Excel letters. Cannot proceed with analysis.")
        print("Please check your Excel column letters and the structure of your DataFrame.")
        print(f"Available DataFrame columns are (first few shown if many): {list(df.columns)[:10]}")
    else:
        print(f"\nSuccessfully mapped. Will analyze these DataFrame columns: {actual_column_names_to_analyze}")

        for col_name in actual_column_names_to_analyze: # Now iterate through the actual names
            print(f"\n\n--- Analysis for Column: {col_name} ---")

            # Column should exist because we just got it from df.columns, but a quick check is fine
            if col_name not in df.columns:
                # This should ideally not happen if the mapping logic above is correct
                print(f"  UNEXPECTED ERROR: Column '{col_name}' not found in the DataFrame after mapping.")
                continue

            column_series = df[col_name]

            if column_series.dropna().empty:
                print("  This column contains no valid data (all NaNs or empty).")
                continue

            # 1. Overall Responses (Value Counts)
            print("\nOverall Responses (Value Counts):")
            overall_responses = column_series.value_counts(dropna=True)
            if overall_responses.empty:
                print("  No non-missing responses found.")
            else:
                overall_df = pd.DataFrame(overall_responses)
                if 'count' in overall_df.columns: # pandas >= 2.0
                    overall_df.rename(columns={'count': 'Count'}, inplace=True)
                elif col_name in overall_df.columns: # older pandas might use original name
                    overall_df.rename(columns={col_name: 'Count'}, inplace=True)
                else: # Fallback if value_counts Series had no name
                     overall_df.columns = ['Count']
                overall_df.index.name = 'Response'
                print(overall_df)

            print("-" * 40)

            # 2. Top 5 Unique Responses (and their percentages)
            print("\nTop 5 Unique Responses (and their percentages):")
            counts = column_series.value_counts(dropna=True)
            percentages = column_series.value_counts(normalize=True, dropna=True) * 100

            if counts.empty:
                print("  No non-missing responses to rank.")
            else:
                top_responses_df = pd.DataFrame({
                    'Response': counts.index,
                    'Count': counts.values,
                    'Percentage (%)': percentages.loc[counts.index].values
                })
                top_5_df = top_responses_df.head(5)
                print(top_5_df.to_string(index=False))

            print("=" * 40)
else:
    print("ERROR: DataFrame 'df' not found or is not a pandas DataFrame.")
    print("Please ensure you have loaded your data into a DataFrame named 'df' in a previous cell.")

Attempting to map Excel column letters to DataFrame column names...
  Excel column 'F' (0-indexed: 5) maps to DataFrame column: 'In which region is the service where you most recently worked as {{APAAA}} based?'
  Excel column 'J' (0-indexed: 9) maps to DataFrame column: 'In which region is the service where you most recently worked with {{PAAAs}} based?'
  Excel column 'DN' (0-indexed: 117) maps to DataFrame column: 'Who was involved in the patient safety incidents that you reported?'
  Excel column 'DO' (0-indexed: 118) maps to DataFrame column: 'Who was involved in the patient safety incidents that you reported?.1'
  Excel column 'DP' (0-indexed: 119) maps to DataFrame column: 'Who have you reported patient safety incidents to?'

Successfully mapped. Will analyze these DataFrame columns: ['In which region is the service where you most recently worked as {{APAAA}} based?', 'In which region is the service where you most recently worked with {{PAAAs}} based?', 'Who was involved in the 

In [16]:
import pandas as pd
from collections import Counter # Useful for counting items in a list

# --- COMPONENT ANALYSIS CHUNK ---
# This chunk assumes 'df' is loaded and 'actual_column_names_to_analyze'
# (list of full column string names) is available from the previous mapping step.

# Example: If you need to re-define/ensure actual_column_names_to_analyze exists for this chunk:
# (This would typically come from the output of the previous Excel-letter-to-column-name mapping)
# For demonstration, let's assume these were the mapped names:
# actual_column_names_to_analyze = [
#     'In which region is the service where you most recently worked as {{APAAA}} based?', # Example for 'F'
#     'When did you qualify as {{APAAA}}?', # Example for 'J'
#     'Job_Roles_PA_Survey_Original', # Example for 'DN'
#     'Job_Roles_AA_Survey_Original', # Example for 'DO'
#     'Healthcare_Settings_PA_Survey_Original' # Example for 'DP'
# ]
# YOU MUST ENSURE THIS LIST IS CORRECTLY POPULATED WITH THE ACTUAL FULL COLUMN NAMES
# FROM YOUR DATAFRAME THAT CORRESPOND TO F, J, DN, DO, DP.

# Number of top components to display
TOP_N_COMPONENTS = 10 # You can change this to 5 or any other number

if 'df' not in locals() or not isinstance(df, pd.DataFrame):
    print("ERROR: DataFrame 'df' not found or is not a pandas DataFrame.")
    print("Please ensure you have loaded your data into 'df' in a previous cell.")
elif 'actual_column_names_to_analyze' not in locals() or not isinstance(actual_column_names_to_analyze, list):
    print("ERROR: 'actual_column_names_to_analyze' list not found.")
    print("Please ensure this list is populated with the target column names from the mapping step.")
else:
    for col_name in actual_column_names_to_analyze:
        print(f"\n\n--- Component Analysis for Column: {col_name} ---")

        if col_name not in df.columns:
            print(f"  ERROR: Column '{col_name}' not found in the DataFrame.")
            continue

        all_individual_components = []
        column_series = df[col_name]

        # Iterate through each response in the column
        for entry in column_series.dropna(): # dropna() to skip missing values
            if pd.isna(entry) or entry == "": # Double check for empty strings even after dropna
                continue

            # Ensure the entry is treated as a string before splitting
            components = str(entry).split('|')
            for component in components:
                cleaned_component = component.strip() # Remove leading/trailing whitespace
                if cleaned_component: # Add only if not an empty string after stripping
                    all_individual_components.append(cleaned_component)

        if not all_individual_components:
            print("  No components found in this column (it might be empty, all NaN, or not use '|' delimiter).")
            continue

        # Count the frequency of each component
        # Option 1: Using collections.Counter
        # component_counts = Counter(all_individual_components)
        # total_components_count = len(all_individual_components)

        # Option 2: Using pandas Series (more consistent with previous outputs)
        component_series = pd.Series(all_individual_components)
        component_value_counts = component_series.value_counts(dropna=False) # Count occurrences of each component
        component_percentages = component_series.value_counts(normalize=True, dropna=False) * 100
        total_individual_selections = len(all_individual_components) # Total number of selections made

        print(f"\nTotal individual selections made in this column: {total_individual_selections}")
        print("Overall Individual Component Counts:")
        if component_value_counts.empty:
            print("  No components to display.")
        else:
            # Create a DataFrame for nice printing
            component_counts_df = pd.DataFrame({
                'Component': component_value_counts.index,
                'Count': component_value_counts.values,
                'Percentage (%)': component_percentages.loc[component_value_counts.index].values
            })
            print(component_counts_df.to_string(index=False))


        print(f"\nTop {TOP_N_COMPONENTS} Individual Components (and their percentages):")
        if component_value_counts.empty:
            print("  No components to display.")
        else:
            # The component_counts_df is already sorted by count
            top_n_df = component_counts_df.head(TOP_N_COMPONENTS)
            print(top_n_df.to_string(index=False))

        print("=" * 50) # Separator for the next column's analysis



--- Component Analysis for Column: In which region is the service where you most recently worked as {{APAAA}} based? ---

Total individual selections made in this column: 1374
Overall Individual Component Counts:
               Component  Count  Percentage (%)
                  London    220       16.011645
      North West England    192       13.973799
      South East England    177       12.882096
      South West England    168       12.227074
Yorkshire and the Humber    141       10.262009
           West Midlands    138       10.043668
                   Wales     88        6.404658
         East of England     86        6.259098
           East Midlands     71        5.167394
      North East England     42        3.056769
                Scotland     35        2.547307
        Northern Ireland     16        1.164483

Top 10 Individual Components (and their percentages):
               Component  Count  Percentage (%)
                  London    220       16.011645
      Nort

In [17]:
import pandas as pd
import numpy as np # For pd.NA

# --- CREATE EXPLODED DATAFRAMES WITH "OTHER" CATEGORY (REVISED THRESHOLD) ---

# This chunk assumes 'df' is loaded and 'actual_column_names_to_analyze'
# (list of full column string names) is available.
# Also, define the name of your Response ID column.
RESPONSE_ID_COLUMN = 'Response ID' # <--- IMPORTANT: Change if your ID column has a different name
OTHER_CATEGORY_NAME = "Other (less than 2%)" # Name for the consolidated category

# Dictionary to store the new DataFrames
exploded_dataframes_final_other = {}

if 'df' not in locals() or not isinstance(df, pd.DataFrame):
    print("ERROR: DataFrame 'df' not found or is not a pandas DataFrame.")
elif RESPONSE_ID_COLUMN not in df.columns:
    print(f"ERROR: Response ID column '{RESPONSE_ID_COLUMN}' not found in the DataFrame.")
    print(f"Available columns: {list(df.columns)}")
elif 'actual_column_names_to_analyze' not in locals() or not isinstance(actual_column_names_to_analyze, list):
    print("ERROR: 'actual_column_names_to_analyze' list not found.")
else:
    for original_col_name in actual_column_names_to_analyze:
        print(f"\n\n--- Processing Column for Exploded DataFrame: {original_col_name} ---")

        if original_col_name not in df.columns:
            print(f"  ERROR: Column '{original_col_name}' not found in the DataFrame. Skipping.")
            continue

        # 1. Create a working DataFrame, select relevant columns, and filter
        # We need the Response ID and the target column, plus any other columns you want to keep
        # For simplicity and to carry all data, let's copy the whole df then filter rows
        working_df = df.copy()
        working_df = working_df[working_df[original_col_name].notna() & (working_df[original_col_name] != '')]

        if working_df.empty:
            print(f"  No valid responses found for column '{original_col_name}' after filtering. Skipping.")
            continue

        # Calculate the number of unique respondents for this question
        unique_respondents_count = working_df[RESPONSE_ID_COLUMN].nunique()
        print(f"  Number of unique respondents for '{original_col_name}': {unique_respondents_count}")

        # 2. Function to split and clean components
        def get_cleaned_components(entry):
            if pd.isna(entry) or entry == '':
                return [pd.NA] # So explode creates a row with NA if original was empty after notna filter
            components = [c.strip() for c in str(entry).split('|') if c.strip()]
            return components if components else [pd.NA] # Handle cases like " | " resulting in empty list

        # Apply splitting
        working_df['temp_split_components'] = working_df[original_col_name].apply(get_cleaned_components)

        # 3. Initial explosion of all components
        # Keep all original columns from working_df
        initial_exploded_df = working_df.explode('temp_split_components')
        
        # Drop rows where the exploded component is NA (resulted from empty or all-space original entries)
        initial_exploded_df = initial_exploded_df.dropna(subset=['temp_split_components'])
        
        # Rename the exploded column
        exploded_col_name_for_analysis = f"{original_col_name}_Individual_Response"
        initial_exploded_df.rename(columns={'temp_split_components': exploded_col_name_for_analysis}, inplace=True)
        
        if initial_exploded_df.empty:
            print(f"  No components found after initial explosion for '{original_col_name}'. Skipping.")
            continue
            
        print(f"  Total individual component selections (rows after initial explode): {len(initial_exploded_df)}")

        # 4. Calculate component frequencies on this initial_exploded_df
        component_counts_in_exploded = initial_exploded_df[exploded_col_name_for_analysis].value_counts(dropna=False)
        total_rows_in_exploded = len(initial_exploded_df) # Sum of component_counts_in_exploded.values

        # 5. Identify components for "Other" (less than 2% of total rows in initial_exploded_df)
        # The threshold is based on the count of rows in the exploded table
        threshold_count_for_other = 0.02 * total_rows_in_exploded
        
        components_to_group_as_other = []
        if total_rows_in_exploded > 0: # Avoid division by zero or issues with empty df
             components_to_group_as_other = component_counts_in_exploded[component_counts_in_exploded < threshold_count_for_other].index.tolist()
             # Ensure "OTHER_CATEGORY_NAME" itself, if it exists as a low-freq original response, isn't grouped into itself.
             # This logic is simpler: we just define OTHER_CATEGORY_NAME as the target.
             # If an original response was LITERALLY "Other (less than 2%)" and was rare, it would become part of this.
        
        final_exploded_df = initial_exploded_df.copy()

        if components_to_group_as_other:
            print(f"  Components to be grouped into '{OTHER_CATEGORY_NAME}' (count < {threshold_count_for_other:.2f}): {components_to_group_as_other}")
            # Replace these low-frequency components with the OTHER_CATEGORY_NAME
            final_exploded_df[exploded_col_name_for_analysis] = final_exploded_df[exploded_col_name_for_analysis].apply(
                lambda x: OTHER_CATEGORY_NAME if x in components_to_group_as_other else x
            )
        else:
            print(f"  No components fall below the 2% threshold (count < {threshold_count_for_other:.2f}) to be grouped into '{OTHER_CATEGORY_NAME}'.")


        # 6. Store the new DataFrame
        # Create a safe name for the dictionary key
        new_df_key = f"exploded_final_{original_col_name.replace(' ', '_').replace('?', '').replace('/', '')[:30]}"
        exploded_dataframes_final_other[new_df_key] = final_exploded_df

        # 7. Report
        print(f"\n  Final Exploded DataFrame '{new_df_key}' created.")
        print(f"  Shape of '{new_df_key}': {final_exploded_df.shape}")

        print(f"\n  Value counts for '{exploded_col_name_for_analysis}' (including '{OTHER_CATEGORY_NAME}'):")
        final_summary_counts = final_exploded_df[exploded_col_name_for_analysis].value_counts(dropna=False)
        final_summary_percentages = final_exploded_df[exploded_col_name_for_analysis].value_counts(normalize=True, dropna=False) * 100
        
        summary_presentation_df = pd.DataFrame({
            'Component': final_summary_counts.index,
            'Count': final_summary_counts.values,
            'Percentage (%)': final_summary_percentages.loc[final_summary_counts.index].values
        })
        print(summary_presentation_df.to_string(index=False))
        print("=" * 70)

# You can now access each new DataFrame from the `exploded_dataframes_final_other` dictionary
# For example, to get one of them:
# if exploded_dataframes_final_other:
#     first_key = list(exploded_dataframes_final_other.keys())[0]
#     print(f"\nFirst 5 rows of '{first_key}':")
#     print(exploded_dataframes_final_other[first_key].head())



--- Processing Column for Exploded DataFrame: In which region is the service where you most recently worked as {{APAAA}} based? ---
  Number of unique respondents for 'In which region is the service where you most recently worked as {{APAAA}} based?': 1362
  Total individual component selections (rows after initial explode): 1374
  Components to be grouped into 'Other (less than 2%)' (count < 27.48): ['Northern Ireland']

  Final Exploded DataFrame 'exploded_final_In_which_region_is_the_service' created.
  Shape of 'exploded_final_In_which_region_is_the_service': (1374, 150)

  Value counts for 'In which region is the service where you most recently worked as {{APAAA}} based?_Individual_Response' (including 'Other (less than 2%)'):
               Component  Count  Percentage (%)
                  London    220       16.011645
      North West England    192       13.973799
      South East England    177       12.882096
      South West England    168       12.227074
Yorkshire and th

In [18]:
import pandas as pd

# --- WRITE EXPLODED DATAFRAMES TO A SINGLE EXCEL FILE ---

# This chunk assumes 'exploded_dataframes_final_other' dictionary is populated
# from the previous step.

output_excel_filename = "exploded_survey_responses_with_other.xlsx"

if 'exploded_dataframes_final_other' not in locals() or not isinstance(exploded_dataframes_final_other, dict):
    print("ERROR: The dictionary 'exploded_dataframes_final_other' was not found.")
    print("Please ensure the previous code chunk that populates this dictionary has been run successfully.")
elif not exploded_dataframes_final_other:
    print("The dictionary 'exploded_dataframes_final_other' is empty. No DataFrames to write.")
else:
    try:
        # Create a Pandas Excel writer using openpyxl as the engine.
        with pd.ExcelWriter(output_excel_filename, engine='openpyxl') as writer:
            print(f"Starting to write DataFrames to '{output_excel_filename}'...")
            for sheet_name_key, df_to_write in exploded_dataframes_final_other.items():
                # Excel sheet names have a maximum length (usually 31 characters)
                # and cannot contain certain characters. Let's create a safe sheet name.
                # The keys in `exploded_dataframes_final_other` are already somewhat sanitized.
                safe_sheet_name = sheet_name_key[:31] # Truncate if too long

                print(f"  Writing DataFrame to sheet: '{safe_sheet_name}'...")
                
                # Write the dataframe to a new sheet
                # index=False will prevent writing the pandas DataFrame index as a column
                df_to_write.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            
        print(f"\nSuccessfully wrote all DataFrames to '{output_excel_filename}'.")
        print("Each DataFrame from 'exploded_dataframes_final_other' is on a separate sheet.")

    except Exception as e:
        print(f"An error occurred while writing to Excel: {e}")
        print("Please ensure you have 'openpyxl' installed (`pip install openpyxl`).")

# To verify, you can list the keys (which became sheet names)
# if 'exploded_dataframes_final_other' in locals() and exploded_dataframes_final_other:
#     print("\nThe following sheets should have been created (keys from the dictionary):")
#     for key_name in exploded_dataframes_final_other.keys():
#         print(f" - {key_name[:31]}")

Starting to write DataFrames to 'exploded_survey_responses_with_other.xlsx'...
  Writing DataFrame to sheet: 'exploded_final_In_which_region_'...
  Writing DataFrame to sheet: 'exploded_final_Who_was_involved'...
  Writing DataFrame to sheet: 'exploded_final_Who_have_you_rep'...

Successfully wrote all DataFrames to 'exploded_survey_responses_with_other.xlsx'.
Each DataFrame from 'exploded_dataframes_final_other' is on a separate sheet.


In [19]:
import pandas as pd
import numpy as np # For pd.NA

# --- FULL PROCESS: MAP EXCEL COLS, EXPLODE WITH 2% OTHER, WRITE TO EXCEL (CORRECTED SHEET NAMES) ---

# Ensure 'df' is your loaded DataFrame from the original Excel file.
# Example:
# file_name = "250409 Leng Survey - Regrouped and Combined.xlsx"
# try:
#     df = pd.read_excel(file_name)
#     print("DataFrame 'df' loaded successfully.")
# except FileNotFoundError:
#     print(f"ERROR: The file '{file_name}' was not found.")
#     df = None
# except Exception as e:
#     print(f"ERROR: An error occurred while loading the file: {e}")
#     df = None

# --- Stage 1: Map Excel Column Letters to Actual DataFrame Column Names ---

# Helper function to convert Excel column letters to 0-based numerical index.
def excel_col_to_index(col_str):
    index = 0
    power = 1
    for char in reversed(col_str.upper()):
        index += (ord(char) - ord('A') + 1) * power
        power *= 26
    return index - 1

excel_column_letters_to_target = ['F', 'J', 'DN', 'DO', 'DP']
# We'll store tuples of (Excel Letter, Actual DataFrame Column Name)
# This helps us use the Excel Letter for the sheet name later.
column_map_to_process = []

if 'df' in locals() and isinstance(df, pd.DataFrame):
    print("Mapping Excel column letters to DataFrame column names...")
    for col_letter in excel_column_letters_to_target:
        try:
            col_index = excel_col_to_index(col_letter)
            if 0 <= col_index < len(df.columns):
                actual_name = df.columns[col_index]
                column_map_to_process.append((col_letter, actual_name)) # Store as tuple
                print(f"  Excel column '{col_letter}' (index {col_index}) maps to DataFrame column: '{actual_name}'")
            else:
                print(f"  ERROR: Excel column '{col_letter}' (index {col_index}) is out of bounds for the DataFrame. "
                      f"DataFrame has {len(df.columns)} columns. Skipping this letter.")
        except Exception as e:
            print(f"  ERROR: Could not process Excel column letter '{col_letter}'. Error: {e}. Skipping this letter.")
    
    if not column_map_to_process:
        print("\nNo valid DataFrame columns were identified from the Excel letters. Cannot proceed.")
else:
    print("ERROR: DataFrame 'df' not found or is not a pandas DataFrame.")
    print("Please ensure you have loaded your data into 'df'.")
    column_map_to_process = [] # Ensure it's empty if df is not found

# --- Stage 2: Process Each Identified Column and Write to Excel ---

RESPONSE_ID_COLUMN = 'Response ID'  # <--- IMPORTANT: Change if your ID column has a different name
OTHER_CATEGORY_NAME = "Other (less than 2%)"
output_excel_filename = "survey_exploded_responses_by_excel_col.xlsx" # Changed filename slightly for clarity

if column_map_to_process: # Only proceed if we have columns to process
    try:
        with pd.ExcelWriter(output_excel_filename, engine='openpyxl') as writer:
            print(f"\nStarting to process columns and write to '{output_excel_filename}'...")

            # Iterate through the list of (Excel Letter, Actual Column Name) tuples
            for excel_letter_for_sheet, original_col_name in column_map_to_process:
                print(f"\n\n--- Processing Data for Excel Sheet: '{excel_letter_for_sheet}' (Original Column: {original_col_name}) ---")

                if original_col_name not in df.columns:
                    print(f"  UNEXPECTED ERROR: Column '{original_col_name}' not found in df. Skipping.")
                    continue
                if RESPONSE_ID_COLUMN not in df.columns:
                    print(f"  ERROR: Response ID column '{RESPONSE_ID_COLUMN}' not found in df. Cannot process '{original_col_name}'.")
                    continue

                # 1. Prepare working DataFrame: copy all columns, filter rows
                working_df_for_col = df.copy()
                working_df_for_col = working_df_for_col[
                    working_df_for_col[original_col_name].notna() & \
                    (working_df_for_col[original_col_name] != '')
                ]

                if working_df_for_col.empty:
                    print(f"  No valid responses found for column '{original_col_name}'. An empty sheet might be created or skipped for sheet '{excel_letter_for_sheet}'.")
                    continue

                unique_respondents_count = working_df_for_col[RESPONSE_ID_COLUMN].nunique()
                print(f"  Number of unique respondents answering this question: {unique_respondents_count}")

                # 2. Split components
                def get_cleaned_components(entry):
                    if pd.isna(entry) or entry == '': return [pd.NA]
                    components = [c.strip() for c in str(entry).split('|') if c.strip()]
                    return components if components else [pd.NA]

                working_df_for_col['temp_split_components'] = working_df_for_col[original_col_name].apply(get_cleaned_components)
                
                # 3. Initial Explode
                initial_exploded_df = working_df_for_col.explode('temp_split_components')
                initial_exploded_df = initial_exploded_df.dropna(subset=['temp_split_components'])

                # Create a unique name for the new exploded column based on the original column name
                # to avoid clashes if the excel_letter is very short.
                exploded_col_name_for_analysis = f"{original_col_name.replace(' ', '_').replace('?', '').replace('/', '')[:20]}_Response"
                initial_exploded_df.rename(columns={'temp_split_components': exploded_col_name_for_analysis}, inplace=True)
                
                if initial_exploded_df.empty:
                    print(f"  No components found after initial explosion for '{original_col_name}'. An empty sheet might be created or skipped for sheet '{excel_letter_for_sheet}'.")
                    continue

                print(f"  Total rows after initial explosion for '{original_col_name}': {len(initial_exploded_df)}")

                # 4. Calculate frequencies and identify components for "Other" (2% threshold)
                component_counts = initial_exploded_df[exploded_col_name_for_analysis].value_counts(dropna=False)
                total_rows_in_exploded = len(initial_exploded_df)
                
                components_to_group_as_other = []
                if total_rows_in_exploded > 0:
                    threshold_count = 0.02 * total_rows_in_exploded
                    components_to_group_as_other = component_counts[component_counts < threshold_count].index.tolist()

                final_processed_df = initial_exploded_df.copy()

                if components_to_group_as_other:
                    print(f"  Components in '{original_col_name}' to be grouped into '{OTHER_CATEGORY_NAME}' (count < {threshold_count:.2f}): {components_to_group_as_other}")
                    final_processed_df[exploded_col_name_for_analysis] = \
                        final_processed_df[exploded_col_name_for_analysis].apply(
                            lambda x: OTHER_CATEGORY_NAME if x in components_to_group_as_other else x
                        )
                else:
                    print(f"  No components in '{original_col_name}' fall below the 2% threshold for '{OTHER_CATEGORY_NAME}'.")
                
                final_processed_df_for_excel = final_processed_df
                
                # 5. Write to Excel sheet USING THE EXCEL LETTER AS THE SHEET NAME
                # Excel letters like 'F', 'J', 'DN' are short and valid sheet names.
                sheet_name_for_excel = excel_letter_for_sheet
                
                print(f"  Writing processed data to sheet: '{sheet_name_for_excel}'")
                final_processed_df_for_excel.to_excel(writer, sheet_name=sheet_name_for_excel, index=False)
                
                # Print summary of the final processed data for this sheet
                print(f"\n  Final value counts for sheet '{sheet_name_for_excel}' (Column: '{exploded_col_name_for_analysis}'):")
                final_summary_counts = final_processed_df_for_excel[exploded_col_name_for_analysis].value_counts(dropna=False)
                final_summary_percentages = final_processed_df_for_excel[exploded_col_name_for_analysis].value_counts(normalize=True, dropna=False) * 100
                summary_df = pd.DataFrame({
                    'Component': final_summary_counts.index,
                    'Count': final_summary_counts.values,
                    'Percentage (%)': final_summary_percentages.loc[final_summary_counts.index].values
                })
                print(summary_df.to_string(index=False))
                print("-" * 50)

            print(f"\nSuccessfully wrote all processed DataFrames to '{output_excel_filename}'.")

    except Exception as e:
        print(f"\nAn error occurred during processing or writing to Excel: {e}")
        print("Please ensure you have 'openpyxl' installed (`pip install openpyxl`).")

elif 'df' in locals() and isinstance(df, pd.DataFrame):
    print("No columns were successfully mapped from Excel letters. Excel file not created.")
# else: df was not found initially, message already printed.

Mapping Excel column letters to DataFrame column names...
  Excel column 'F' (index 5) maps to DataFrame column: 'In which region is the service where you most recently worked as {{APAAA}} based?'
  Excel column 'J' (index 9) maps to DataFrame column: 'In which region is the service where you most recently worked with {{PAAAs}} based?'
  Excel column 'DN' (index 117) maps to DataFrame column: 'Who was involved in the patient safety incidents that you reported?'
  Excel column 'DO' (index 118) maps to DataFrame column: 'Who was involved in the patient safety incidents that you reported?.1'
  Excel column 'DP' (index 119) maps to DataFrame column: 'Who have you reported patient safety incidents to?'

Starting to process columns and write to 'survey_exploded_responses_by_excel_col.xlsx'...


--- Processing Data for Excel Sheet: 'F' (Original Column: In which region is the service where you most recently worked as {{APAAA}} based?) ---
  Number of unique respondents answering this question

In [20]:
import pandas as pd

# --- VERIFICATION CHUNK: UNIQUE IDs AND ROW COUNTS FROM EXCEL SHEETS ---

# Ensure 'df' (original DataFrame) is loaded and 'column_map_to_process'
# and 'output_excel_filename' are available from the previous main script.
# Also, RESPONSE_ID_COLUMN needs to be defined as it was in the main script.

# These should match the values from your main processing script:
# If they are not in scope, you'll need to define them or ensure they are.
# For example:
# df = pd.read_excel("250409 Leng Survey - Regrouped and Combined.xlsx") # Or however it was loaded
# column_map_to_process = [('F', 'Actual Col Name for F'), ('J', 'Actual Col Name for J'), ...] # From mapping
# output_excel_filename = "survey_exploded_responses_by_excel_col.xlsx"
# RESPONSE_ID_COLUMN = 'Response ID'

print("--- Verification of Exploded Data ---")

# 1. Total Unique Response IDs in the original DataFrame
if 'df' in locals() and isinstance(df, pd.DataFrame) and RESPONSE_ID_COLUMN in df.columns:
    total_unique_ids_original_df = df[RESPONSE_ID_COLUMN].nunique()
    print(f"\nTotal unique '{RESPONSE_ID_COLUMN}' values in the original DataFrame: {total_unique_ids_original_df}")
else:
    print(f"\nCould not calculate total unique '{RESPONSE_ID_COLUMN}'s from original 'df'. "
          "Ensure 'df' is loaded and contains the ID column.")

# 2. Row counts and unique IDs for each sheet in the created Excel file
if 'output_excel_filename' not in locals():
    print("\nERROR: 'output_excel_filename' is not defined. Cannot read from the Excel file.")
elif 'column_map_to_process' not in locals() or not isinstance(column_map_to_process, list) or not column_map_to_process:
    print("\nERROR: 'column_map_to_process' is not defined or is empty. Cannot determine which sheets to check.")
else:
    print(f"\n--- Analyzing sheets from Excel file: '{output_excel_filename}' ---")
    all_sheets_data = {}
    try:
        # Read all sheets from the Excel file into a dictionary of DataFrames
        # Setting sheet_name=None reads all sheets
        all_sheets_data = pd.read_excel(output_excel_filename, sheet_name=None)
        print(f"Successfully read {len(all_sheets_data)} sheets from the Excel file.")
    except FileNotFoundError:
        print(f"ERROR: The Excel file '{output_excel_filename}' was not found. Please ensure the main script ran successfully.")
    except Exception as e:
        print(f"ERROR: An error occurred while reading '{output_excel_filename}': {e}")

    if all_sheets_data:
        # Iterate through the mappings we used to create the sheets
        for excel_letter_sheet_name, original_df_col_name in column_map_to_process:
            print(f"\n--- Sheet: '{excel_letter_sheet_name}' (Corresponds to original column: '{original_df_col_name}') ---")

            if excel_letter_sheet_name in all_sheets_data:
                sheet_df = all_sheets_data[excel_letter_sheet_name]
                
                # Total row count for this sheet
                row_count_sheet = len(sheet_df)
                print(f"  Total rows in this sheet: {row_count_sheet}")

                # Number of unique Response IDs in this sheet
                if RESPONSE_ID_COLUMN in sheet_df.columns:
                    unique_ids_in_sheet = sheet_df[RESPONSE_ID_COLUMN].nunique()
                    print(f"  Number of unique '{RESPONSE_ID_COLUMN}' values in this sheet: {unique_ids_in_sheet}")
                else:
                    print(f"  WARNING: Column '{RESPONSE_ID_COLUMN}' not found in sheet '{excel_letter_sheet_name}'. Cannot count unique IDs for this sheet.")
            else:
                print(f"  WARNING: Sheet '{excel_letter_sheet_name}' not found in the Excel file '{output_excel_filename}'.")
    elif 'output_excel_filename' in locals(): # File might exist but no sheets read (e.g. empty file)
        print("No sheets were read from the Excel file, or the file was empty/corrupted.")

--- Verification of Exploded Data ---

Total unique 'Response ID' values in the original DataFrame: 8558

--- Analyzing sheets from Excel file: 'survey_exploded_responses_by_excel_col.xlsx' ---
Successfully read 5 sheets from the Excel file.

--- Sheet: 'F' (Corresponds to original column: 'In which region is the service where you most recently worked as {{APAAA}} based?') ---
  Total rows in this sheet: 1374
  Number of unique 'Response ID' values in this sheet: 1362

--- Sheet: 'J' (Corresponds to original column: 'In which region is the service where you most recently worked with {{PAAAs}} based?') ---
  Total rows in this sheet: 7562
  Number of unique 'Response ID' values in this sheet: 7287

--- Sheet: 'DN' (Corresponds to original column: 'Who was involved in the patient safety incidents that you reported?') ---
  Total rows in this sheet: 1039
  Number of unique 'Response ID' values in this sheet: 519

--- Sheet: 'DO' (Corresponds to original column: 'Who was involved in the pa